In [187]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import pandas as pd
import selenium
import os
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np
import json
from pprint import pprint
from sqlalchemy import create_engine,inspect, func


# Connect to chromedriver and BS

In [10]:
executable_path = {'executable_path': '/Users/hmm794/Downloads/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'http://www.dph.illinois.gov/data-statistics/vital-statistics/illinois-population-data'
browser.visit(url)

# EXTRACT DATA

# Scrape Data

In [11]:
html = browser.html
soup = bs(html, 'html.parser')
soup.findAll('tr', limit=1)

[<tr><th>
 <p>County</p>
 </th>
 <th>April 1, 1980<br/>Census</th>
 <th>April 1, 1990<br/>Census</th>
 <th>April 1, 2000<br/>Census</th>
 <th>April 1, 2010<br/>Census</th>
 <th>Percent Change<br/>1980 to 1990</th>
 <th>Percent Change<br/>1990 to 2000</th>
 <th>Percent Change<br/>2000 to 2010</th>
 </tr>]

In [12]:
#test to see if the code works- 
soup.findAll('tr', limit=1)[0].findAll('th') 

[<th>
 <p>County</p>
 </th>,
 <th>April 1, 1980<br/>Census</th>,
 <th>April 1, 1990<br/>Census</th>,
 <th>April 1, 2000<br/>Census</th>,
 <th>April 1, 2010<br/>Census</th>,
 <th>Percent Change<br/>1980 to 1990</th>,
 <th>Percent Change<br/>1990 to 2000</th>,
 <th>Percent Change<br/>2000 to 2010</th>]

In [86]:
#get column headers
column_headers = [th.getText() for th in 
                  soup.findAll('tr', limit=1)[0].findAll('th')]
column_headers

['\nCounty\n',
 'April 1, 1980Census',
 'April 1, 1990Census',
 'April 1, 2000Census',
 'April 1, 2010Census',
 'Percent Change1980 to 1990',
 'Percent Change1990 to 2000',
 'Percent Change2000 to 2010']

In [87]:
column_headers[column_headers.index('\nCounty\n')] = 'County'
print(column_headers)

['County', 'April 1, 1980Census', 'April 1, 1990Census', 'April 1, 2000Census', 'April 1, 2010Census', 'Percent Change1980 to 1990', 'Percent Change1990 to 2000', 'Percent Change2000 to 2010']


In [88]:
data_rows = soup.findAll('tr')[1:]  # skip the first  header rows

In [89]:
type(data_rows)  # now we have a list of table rows

list

In [90]:
census = [[td.getText() for td in data_rows[i].findAll('td')]
            for i in range(len(data_rows))]

In [105]:
df = pd.DataFrame(census,columns=column_headers)
df.head(5)

,County,"April 1, 1980Census","April 1, 1990Census","April 1, 2000Census","April 1, 2010Census",Percent Change1980 to 1990,Percent Change1990 to 2000,Percent Change2000 to 2010
0,Illinois,"\n11,426,518\n","\n11,430,602\n","\n12,419,293\n","\n12,830,632\n",\n0.04\n,\n8.6\n,\n3.3\n
1,Adams,"\n71,622\n","\n66,090\n","\n68,277\n","\n67,103\n",\n-7.7\n,\n3.3\n,\n-1.7\n
2,Alexander,"\n12,264\n","\n10,626\n","\n9,590\n","\n8,238\n",\n-13.4\n,\n-9.7\n,\n-14.1\n
3,Bond,"\n16,224\n","\n14,991\n","\n17,633\n","\n17,768\n",\n-7.6\n,\n17.6\n,\n0.8\n
4,Boone,"\n28,630\n","\n30,806\n","\n41,786\n","\n54,165\n",\n7.6\n,\n35.6\n,\n29.6\n


In [106]:
df.dtypes

County                        object
April 1, 1980Census           object
April 1, 1990Census           object
April 1, 2000Census           object
April 1, 2010Census           object
Percent Change1980 to 1990    object
Percent Change1990 to 2000    object
Percent Change2000 to 2010    object
dtype: object

# TRANSFORM

In [107]:
# df['April 1, 1980Census']= df['April 1, 1980Census'].astype(float)
df['April 1, 1980Census'] = df['April 1, 1980Census'].str.replace(r'\D', '')
df['April 1, 1990Census'] = df['April 1, 1990Census'].str.replace(r'\D', '')
df['April 1, 2000Census'] = df['April 1, 2000Census'].str.replace(r'\D', '')
df['April 1, 2010Census'] = df['April 1, 2010Census'].str.replace(r'\D', '')
df['Percent Change1980 to 1990'] = df['Percent Change1980 to 1990'].str.replace(r'\D', '')
df['Percent Change1990 to 2000'] = df['Percent Change1990 to 2000'].str.replace(r'\D', '')
df['Percent Change2000 to 2010'] = df['Percent Change2000 to 2010'].str.replace(r'\D', '')

In [108]:
df.head(5)

,County,"April 1, 1980Census","April 1, 1990Census","April 1, 2000Census","April 1, 2010Census",Percent Change1980 to 1990,Percent Change1990 to 2000,Percent Change2000 to 2010
0,Illinois,11426518,11430602,12419293,12830632,004,86,33
1,Adams,71622,66090,68277,67103,77,33,17
2,Alexander,12264,10626,9590,8238,134,97,141
3,Bond,16224,14991,17633,17768,76,176,08
4,Boone,28630,30806,41786,54165,76,356,296


In [110]:
f = df.apply(pd.to_numeric, errors='ignore')
f.dtypes

County                        object
April 1, 1980Census            int64
April 1, 1990Census            int64
April 1, 2000Census            int64
April 1, 2010Census            int64
Percent Change1980 to 1990     int64
Percent Change1990 to 2000     int64
Percent Change2000 to 2010     int64
dtype: object

,County,"April 1, 1980Census","April 1, 1990Census","April 1, 2000Census","April 1, 2010Census",Percent Change1980 to 1990,Percent Change1990 to 2000,Percent Change2000 to 2010
0,Illinois,11426518,11430602,12419293,12830632,4,86,33
1,Adams,71622,66090,68277,67103,77,33,17
2,Alexander,12264,10626,9590,8238,134,97,141
3,Bond,16224,14991,17633,17768,76,176,8
4,Boone,28630,30806,41786,54165,76,356,296


In [154]:
f.reset_index(level=0, inplace=True)
f.head(5)

,level_0,index,County,"April 1, 1980Census","April 1, 1990Census","April 1, 2000Census","April 1, 2010Census",Percent Change1980 to 1990,Percent Change1990 to 2000,Percent Change2000 to 2010
0,0,0,Illinois,11426518,11430602,12419293,12830632,4,86,33
1,1,1,Adams,71622,66090,68277,67103,77,33,17
2,2,2,Alexander,12264,10626,9590,8238,134,97,141
3,3,3,Bond,16224,14991,17633,17768,76,176,8
4,4,4,Boone,28630,30806,41786,54165,76,356,296


In [172]:
illinois_pop = f[['index','County', 'April 1, 2010Census', 'Percent Change1980 to 1990', 'Percent Change1990 to 2000','Percent Change2000 to 2010']]
illinois_pop = illinois_pop.rename(columns={"index":'id','April 1, 2010Census':'April_count','Percent Change1980 to 1990':'Percent_change_80_90','Percent Change1990 to 2000':'Percent_change_90_00','Percent Change2000 to 2010':'Percent_change_00_10'})
illinois_pop.head(10)

,id,County,April_count,Percent_change_80_90,Percent_change_90_00,Percent_change_00_10
0,0,Illinois,12830632,4,86,33
1,1,Adams,67103,77,33,17
2,2,Alexander,8238,134,97,141
3,3,Bond,17768,76,176,8
4,4,Boone,54165,76,356,296
5,5,Brown,6937,79,191,2
6,6,Bureau,34978,88,5,15
7,7,Calhoun,5089,93,45,1
8,8,Carroll,15387,105,8,77
9,9,Cass,13642,109,19,4


In [157]:
illinois_pop.dtypes

id                             int64
County                        object
April 1, 2010Census            int64
Percent Change1980 to 1990     int64
Percent Change1990 to 2000     int64
Percent Change2000 to 2010     int64
dtype: object

# extract data for second table

In [158]:
df2 = pd.read_csv("https://www.ers.usda.gov/webdocs/DataFiles/48747/Unemployment.csv?v=9115.7")

employ = df2[['FIPS', 'State', 'Area_name', 'Civilian_labor_force_2017', 'Employed_2017', 'Unemployed_2017',
       'Unemployment_rate_2017']]
employ.head(5)

,FIPS,State,Area_name,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017
0,0,US,United States,"160,273,906","153,293,830","6,980,076",4.4
1,1000,AL,Alabama,2178243,2081676,96567,4.4
2,1001,AL,"Autauga County, AL",25972,24958,1014,3.9
3,1003,AL,"Baldwin County, AL",92090,88342,3748,4.1
4,1005,AL,"Barbour County, AL",8317,7831,486,5.8


# extract data for third table

In [159]:
df3 = pd.read_csv('https://www.ers.usda.gov/webdocs/DataFiles/48747/PovertyEstimates.csv?v=2529')

broke = df3[['FIPStxt', 'POVALL_2017', 'PCTPOVALL_2017', 'MEDHHINC_2017']]

poor = pov.rename(columns = {'FIPStxt': 'FIPS', 'POVALL_2017': 'Total_poverty_2017', 'PCTPOVALL_2017': 'Poverty_rate_2017',
                               'MEDHHINC_2017': 'Median_income_2017'})

poor.head(5)

,FIPS,Total_poverty_2017,Poverty_rate_2017,Median_income_2017
0,0,"42,583,651",13.4,"60,336"
1,1000,"802,263",16.9,"48,193"
2,1001,"7,390",13.4,"58,343"
3,1003,"21,199",10.1,"56,607"
4,1005,"7,414",33.4,"32,490"


# TRANSFORM DATA


# Clean up and merge tables

In [160]:
povertydf = pd.merge(employ, poor, how = 'inner', on = 'FIPS', validate = '1:1')
povertydf.head(5)

,FIPS,State,Area_name,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Total_poverty_2017,Poverty_rate_2017,Median_income_2017
0,0,US,United States,"160,273,906","153,293,830","6,980,076",4.4,"42,583,651",13.4,"60,336"
1,1000,AL,Alabama,2178243,2081676,96567,4.4,"802,263",16.9,"48,193"
2,1001,AL,"Autauga County, AL",25972,24958,1014,3.9,"7,390",13.4,"58,343"
3,1003,AL,"Baldwin County, AL",92090,88342,3748,4.1,"21,199",10.1,"56,607"
4,1005,AL,"Barbour County, AL",8317,7831,486,5.8,"7,414",33.4,"32,490"


In [161]:
IL_poverty = povertydf.loc[povertydf['State'] == 'IL']
IL_poverty.head(5)

,FIPS,State,Area_name,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Total_poverty_2017,Poverty_rate_2017,Median_income_2017
608,17000,IL,Illinois,6489612,6170546,319066,4.9,"1,568,670",12.5,"63,044"
609,17001,IL,"Adams County, IL",32386,31112,1274,3.9,"8,608",13.5,"49,088"
610,17003,IL,"Alexander County, IL",2131,1953,178,8.4,"1,889",30.3,"33,985"
611,17005,IL,"Bond County, IL",7919,7565,354,4.5,"1,993",13.3,"54,831"
612,17007,IL,"Boone County, IL",26220,24448,1772,6.8,"4,676",8.8,"69,413"


In [162]:
IL_poverty2 = IL_poverty.rename(columns={"FIPS":'id'})
IL_poverty2['Median_income_2017'] = IL_poverty2['Median_income_2017'].str.replace(',', '')
IL_poverty2['Total_poverty_2017'] = IL_poverty2['Total_poverty_2017'].str.replace(',', '')
IL_poverty2['Unemployment_rate_2017'] = IL_poverty2['Unemployment_rate_2017'].astype(int)
IL_poverty2['Poverty_rate_2017'] = IL_poverty2['Poverty_rate_2017'].astype(int)
IL_poverty3 = IL_poverty2.apply(pd.to_numeric, errors='ignore')
IL_poverty3.head(5)
IL_poverty3.dtypes


id                            int64
State                        object
Area_name                    object
Civilian_labor_force_2017     int64
Employed_2017                 int64
Unemployed_2017               int64
Unemployment_rate_2017        int64
Total_poverty_2017            int64
Poverty_rate_2017             int64
Median_income_2017            int64
dtype: object

# export data as csv

In [163]:
poverty.to_csv('Poverty_data.csv', index=False, header=True)
illinois_pop.to_csv('Illinois_Census_2010_only.csv', index=True, header=True)
IL_poverty3.to_csv('Illinois_poverty_data_2017.csv', index=False, header=True)

# LOAD

# Connect to PG Admin server

In [179]:
password = "Radeke23@"
rds_connection_string = f'postgres:{password}@localhost:5432/ETL-Project_MCS2019'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [180]:
engine.table_names()

['illinois_poverty', 'illinois_census']

# Check the columns in your tables

In [181]:
pd.read_sql_query('select * from illinois_poverty', con=engine).head()

,id,State,Area_name,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Total_poverty_2017,Poverty_rate_2017,Median_income_2017


In [182]:
pd.read_sql_query('select * from illinois_census', con=engine).head()

,id,County,April_count,Percent_change_80_90,Percent_change_90_00,Percent_change_00_10


### Use pandas to load csv converted DataFrame into database

In [183]:
IL_poverty3.to_sql(name='illinois_poverty', con=engine, if_exists='append', index=False)

In [184]:
pd.read_sql_query('select * from illinois_poverty', con=engine).head(5)

,id,State,Area_name,Civilian_labor_force_2017,Employed_2017,Unemployed_2017,Unemployment_rate_2017,Total_poverty_2017,Poverty_rate_2017,Median_income_2017
0,17000,IL,Illinois,6489612,6170546,319066,4,1568670,12,63044
1,17001,IL,"Adams County, IL",32386,31112,1274,3,8608,13,49088
2,17003,IL,"Alexander County, IL",2131,1953,178,8,1889,30,33985
3,17005,IL,"Bond County, IL",7919,7565,354,4,1993,13,54831
4,17007,IL,"Boone County, IL",26220,24448,1772,6,4676,8,69413


In [185]:
illinois_pop.to_sql(name='illinois_census', con=engine, if_exists='append', index=False)

In [186]:
pd.read_sql_query('select * from illinois_census', con=engine).head(5)

,id,County,April_count,Percent_change_80_90,Percent_change_90_00,Percent_change_00_10
0,0,Illinois,12830632,4,86,33
1,1,Adams,67103,77,33,17
2,2,Alexander,8238,134,97,141
3,3,Bond,17768,76,176,8
4,4,Boone,54165,76,356,296


In [258]:
import matplotlib
from matplotlib import style
style.use('seaborn')
import matplotlib.pyplot as plt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
  

In [259]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['illinois_poverty', 'illinois_census']

In [260]:
IP = Base.classes.illinois_poverty

In [261]:
IC = Base.classes.illinois_census

In [262]:
session = Session(engine)

In [263]:
inspector = inspect(engine)
inspector.get_table_names()

['illinois_poverty', 'illinois_census']

In [264]:
# Get a list of column names and types
columns = inspector.get_columns('illinois_poverty')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
State TEXT
Area_name TEXT
Civilian_labor_force_2017 INTEGER
Employed_2017 INTEGER
Unemployed_2017 INTEGER
Unemployment_rate_2017 INTEGER
Total_poverty_2017 INTEGER
Poverty_rate_2017 INTEGER
Median_income_2017 INTEGER


In [265]:
# Get a list of column names and types
columns = inspector.get_columns('illinois_census')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
County TEXT
April_count INTEGER
Percent_change_80_90 INTEGER
Percent_change_90_00 INTEGER
Percent_change_00_10 INTEGER


In [266]:
session.query(IP.Area_name, IC.County).limit(300).all()

[('Illinois', 'Illinois'),
 ('Illinois', 'Adams'),
 ('Illinois', 'Alexander'),
 ('Illinois', 'Bond'),
 ('Illinois', 'Boone'),
 ('Illinois', 'Brown'),
 ('Illinois', 'Bureau'),
 ('Illinois', 'Calhoun'),
 ('Illinois', 'Carroll'),
 ('Illinois', 'Cass'),
 ('Illinois', 'Champaign'),
 ('Illinois', 'Christian'),
 ('Illinois', 'Clark'),
 ('Illinois', 'Clay'),
 ('Illinois', 'Clinton'),
 ('Illinois', 'Coles'),
 ('Illinois', 'Cook'),
 ('Illinois', 'Crawford'),
 ('Illinois', 'Cumberland'),
 ('Illinois', 'DeKalb'),
 ('Illinois', 'De Witt'),
 ('Illinois', 'Douglas'),
 ('Illinois', 'DuPage'),
 ('Illinois', 'Edgar'),
 ('Illinois', 'Edwards'),
 ('Illinois', 'Effingham'),
 ('Illinois', 'Fayette'),
 ('Illinois', 'Ford'),
 ('Illinois', 'Franklin'),
 ('Illinois', 'Fulton'),
 ('Illinois', 'Gallatin'),
 ('Illinois', 'Greene'),
 ('Illinois', 'Grundy'),
 ('Illinois', 'Hamilton'),
 ('Illinois', 'Hancock'),
 ('Illinois', 'Hardin'),
 ('Illinois', 'Henderson'),
 ('Illinois', 'Henry'),
 ('Illinois', 'Iroquois'),
 ('

In [267]:
same_county= session.query(IP, IC).filter(IP.Area_name == IC.County).limit(300).all()


for record in same_county:
    (IP, IC) = record
    print(IP.Area_name)
    print(IC.County)

Illinois
Illinois
